## Import

In [1]:
import warnings
warnings.filterwarnings(action = 'ignore')

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [2]:
import Basic_model_MS_LapSRN as Basic_model
from utils import psnr
from utils import ssim
from utils import adjust_learning_rate
import prepare_sub_image_Three_inputs as ps
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
import tensorflow as tf
import cv2
import numpy as np
from scipy import io
import keras.backend as K
from collections import defaultdict
import time
import pandas as pd

Using TensorFlow backend.


## Test

In [5]:
scale = 2
test_set = 'Urban100'
conv_side = 8
alpha = 0.5
D = 5
R = 8


def test():
    
    model4 = Basic_model.MS_LapSRN_model(D = D, R = R)
    
    model4.compile(optimizer = 'adam', loss = 'mean_absolute_error')
    
    model4.load_weights('checkpoint/MS_LapSRN_DIV2K_alpha:{}_2.h5'.format(alpha))
    
    print(model4.summary())

    PATH_image = '../../Dataset/MS_LapSRN/Test/{}/'.format(test_set)
    IMAGE_PATH_x2 = 'output_image/{}/scale{}'.format(test_set, scale)
    IMAGE_PATH_x4 = 'output_image/{}/scale{}'.format(test_set, scale*2)
    
    names_image = os.listdir(PATH_image)
    names_image = sorted(names_image)
    
    nums = len(names_image)
    
    count = 0
    global total_history
    
    
    for i in range(0, 1):

        psnr_model4_x2 = []
        psnr_bicubic_x2 = []
        
        psnr_model4_x4 = []
        psnr_bicubic_x4 = []
        
        ssim_model4_x2 = []
        ssim_bicubic_x2 = []
        
        ssim_model4_x4 = []
        ssim_bicubic_x4 = []
        
        total_time = []
        
        
        for i in range(nums):
            
            
            OUTPUT_NAME_x2 = IMAGE_PATH_x2 + '/' + 'MS_LapSRN_alpha{}_x2_{}.png'.format(alpha, i)
            OUTPUT_NAME_x4 = IMAGE_PATH_x4 + '/' + 'MS_LapSRN_alpha{}_x4_{}.png'.format(alpha, i)

            mat_image = io.loadmat(PATH_image + names_image[i])
            
            input_img = mat_image['im_input_rgb']
            
            hr_img_x2 = mat_image['im_hr_x2_rgb']
            bicubic_img_x2 = mat_image['im_bicubic_x2_rgb']
            
            hr_img_x4 = mat_image['im_hr_x4_rgb']
            bicubic_img_x4 = mat_image['im_bicubic_x4_rgb']
            
            shape_input = input_img.shape
            shape_x2 = hr_img_x2.shape
            shape_x4 = hr_img_x4.shape
            
            input_RGB = np.zeros([1, shape_input[0], shape_input[1], 3])
            input_RGB[0, :, :, :] = input_img / 255
            
            start_time = time.time()

            pre = model4.predict(input_RGB, batch_size = 1)
            pre_x2 = pre[0]
            pre_x4 = pre[1]
            
            pre_x2 = pre_x2 * 255
            pre_x4 = pre_x4 * 255
            
            pre_x2[pre_x2[:] > 255] = 255
            pre_x2[pre_x2[:] < 0] = 0
            pre_x4[pre_x4[:] > 255] = 255
            pre_x4[pre_x4[:] < 0] = 0
            
            finish_time = time.time()
            
            output_img_x2 = np.zeros([shape_x2[0], shape_x2[1], 3])
            output_img_x2[:, :, 2] = pre_x2[0, :, :, 0]
            output_img_x2[:, :, 1] = pre_x2[0, :, :, 1]
            output_img_x2[:, :, 0] = pre_x2[0, :, :, 2]
            
            cv2.imwrite(OUTPUT_NAME_x2, output_img_x2)
            
            hr_img_x2_r = hr_img_x2[:, :, 0]
            hr_img_x2_g = hr_img_x2[:, :, 1]
            hr_img_x2_b = hr_img_x2[:, :, 2]
            
            output_img_x2_r = output_img_x2[:, :, 2]
            output_img_x2_g = output_img_x2[:, :, 1]
            output_img_x2_b = output_img_x2[:, :, 0]
            
            bicubic_img_x2_r = bicubic_img_x2[:, :, 0]
            bicubic_img_x2_g = bicubic_img_x2[:, :, 1]
            bicubic_img_x2_b = bicubic_img_x2[:, :, 2]
            
            hr_img_x2_Y = 16 + (65.738 * hr_img_x2_r + 129.057 * hr_img_x2_g + 25.064 * hr_img_x2_b) / 255
            output_img_x2_Y = 16 + (65.738 * output_img_x2_r + 129.057 * output_img_x2_g + 25.064 * output_img_x2_b) / 255
            bicubic_img_x2_Y = 16 + (65.738 * bicubic_img_x2_r + 129.057 * bicubic_img_x2_g + 25.064 * bicubic_img_x2_b) / 255
            
            
            output_img_x4 = np.zeros([shape_x4[0], shape_x4[1], 3])
            output_img_x4[:, :, 2] = pre_x4[0, :, :, 0]
            output_img_x4[:, :, 1] = pre_x4[0, :, :, 1]
            output_img_x4[:, :, 0] = pre_x4[0, :, :, 2]
            
            cv2.imwrite(OUTPUT_NAME_x4, output_img_x4)
            
            hr_img_x4_r = hr_img_x4[:, :, 0]
            hr_img_x4_g = hr_img_x4[:, :, 1]
            hr_img_x4_b = hr_img_x4[:, :, 2]
            
            output_img_x4_r = output_img_x4[:, :, 2]
            output_img_x4_g = output_img_x4[:, :, 1]
            output_img_x4_b = output_img_x4[:, :, 0]
            
            bicubic_img_x4_r = bicubic_img_x4[:, :, 0]
            bicubic_img_x4_g = bicubic_img_x4[:, :, 1]
            bicubic_img_x4_b = bicubic_img_x4[:, :, 2]
            
            hr_img_x4_Y = 16 + (65.738 * hr_img_x4_r + 129.057 * hr_img_x4_g + 25.064 * hr_img_x4_b) / 255
            output_img_x4_Y = 16 + (65.738 * output_img_x4_r + 129.057 * output_img_x4_g + 25.064 * output_img_x4_b) / 255 
            bicubic_img_x4_Y = 16 + (65.738 * bicubic_img_x4_r + 129.057 * bicubic_img_x4_g + 25.064 * bicubic_img_x4_b) / 255
            
            
            # YCrCb Channel에서 Y에 대해 PSNR 측정 시
            hr_img_x2_measure = hr_img_x2_Y[conv_side:-conv_side, conv_side:-conv_side]
            output_img_x2_measure = output_img_x2_Y[conv_side:-conv_side, conv_side:-conv_side]
            bicubic_img_x2_measure = bicubic_img_x2_Y[conv_side:-conv_side, conv_side:-conv_side]
            
            psnr_x2 = psnr(output_img_x2_measure, hr_img_x2_measure)
            ssim_x2 = ssim(output_img_x2_measure, hr_img_x2_measure)
            
            psnr_x2_bicubic = psnr(bicubic_img_x2_measure, hr_img_x2_measure)
            ssim_x2_bicubic = ssim(bicubic_img_x2_measure, hr_img_x2_measure)
            
            hr_img_x4_measure = hr_img_x4_Y[conv_side:-conv_side, conv_side:-conv_side]
            output_img_x4_measure = output_img_x4_Y[conv_side:-conv_side, conv_side:-conv_side]
            bicubic_img_x4_measure = bicubic_img_x4_Y[conv_side:-conv_side, conv_side:-conv_side]
            
            psnr_x4 = psnr(output_img_x4_measure, hr_img_x4_measure)
            ssim_x4 = ssim(output_img_x4_measure, hr_img_x4_measure)
            
            psnr_x4_bicubic = psnr(bicubic_img_x4_measure, hr_img_x4_measure)
            ssim_x4_bicubic = ssim(bicubic_img_x4_measure, hr_img_x4_measure)
            
            print(i + 1)
            
            print('Bicubic_x2: ', psnr_x2_bicubic, 'ssim: ', ssim_x2_bicubic)
            print('Model4_x2: ', psnr_x2, 'ssim: ', ssim_x2)
            
            print('Bicubic_x4: ', psnr_x4_bicubic, 'ssim: ', ssim_x4_bicubic)
            print('Model4_x4: ', psnr_x4, 'ssim: ', ssim_x4)
            
            time_network = finish_time - start_time
            print('time: ', time_network)
                        
            
            psnr_bicubic_x2.append(psnr_x2_bicubic)
            ssim_bicubic_x2.append(ssim_x2_bicubic)
            
            psnr_model4_x2.append(psnr_x2)
            ssim_model4_x2.append(ssim_x2)
            
            psnr_bicubic_x4.append(psnr_x4_bicubic)
            ssim_bicubic_x4.append(ssim_x4_bicubic)
            
            psnr_model4_x4.append(psnr_x4)
            ssim_model4_x4.append(ssim_x4)
            
            total_time.append(time_network)
            
            
        model_x2_dataframe = pd.DataFrame(psnr_model4_x2)
        model_x4_dataframe = pd.DataFrame(psnr_model4_x4)

        model_x4_dataframe.to_csv('excel/{}_x4_D{}R{}_alpha:{}.csv'.format(test_set, D, R, alpha), header = ['model_alpha:{}'.format(alpha)], index = True)
        
        psnr_bicubic_x2_final = np.mean(psnr_bicubic_x2)
        ssim_bicubic_x2_final = np.mean(ssim_bicubic_x2)
        
        psnr_model4_x2_final = np.mean(psnr_model4_x2)
        ssim_model4_x2_final = np.mean(ssim_model4_x2)
        
        
        psnr_bicubic_x4_final = np.mean(psnr_bicubic_x4)
        ssim_bicubic_x4_final = np.mean(ssim_bicubic_x4)
        
        psnr_model4_x4_final = np.mean(psnr_model4_x4)
        ssim_model4_x4_final = np.mean(ssim_model4_x4)
        
        time_final = np.sum(total_time)
        
        print('-------------------------------')
        print('Time: ', time_final)
        print('------------------------------')
        
        
        print('Bicubic_x2')
        print('PSNR: ', psnr_bicubic_x2_final, 'SSIM: ', ssim_bicubic_x2_final)
        print('Model4_x2')
        print('PSNR: ', psnr_model4_x2_final, 'SSIM: ', ssim_model4_x2_final)
        
        print('Bicubic_x4')
        print('PSNR: ', psnr_bicubic_x4_final, 'SSIM: ', ssim_bicubic_x4_final)
        print('Model4_x4')
        print('PSNR: ', psnr_model4_x4_final, 'SSIM: ', ssim_model4_x4_final)
        
        tf.keras.backend.clear_session()
        
    
if __name__ == '__main__':
    
    test()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
init_conv (Model)               (None, None, None, 6 1792        input_6[0][0]                    
__________________________________________________________________________________________________
Embedding (Model)               (None, None, None, 6 250240      init_conv[1][0]                  
                                                                 Embedding[1][0]                  
__________________________________________________________________________________________________
upsample (Model)                (None, None, None, 3 147         input_6[0][0]                

24
Bicubic_x2:  26.19235878349125 ssim:  0.6357360684087575
Model4_x2:  29.219881748114823 ssim:  0.7614179642410528
Bicubic_x4:  21.958219082172917 ssim:  0.299660569373378
Model4_x4:  23.4325320164086 ssim:  0.3976085571480117
time:  0.13383865356445312
25
Bicubic_x2:  24.660579114938507 ssim:  0.6484642716910607
Model4_x2:  28.447193344944267 ssim:  0.8014775822037824
Bicubic_x4:  21.569227635312895 ssim:  0.3504803246487167
Model4_x4:  23.77254183552843 ssim:  0.47741565962158666
time:  0.4954259395599365
26
Bicubic_x2:  26.66931344628335 ssim:  0.6533720670577398
Model4_x2:  28.961684623940553 ssim:  0.779033310482169
Bicubic_x4:  22.77316132304471 ssim:  0.3336492590593731
Model4_x4:  23.835788151860978 ssim:  0.4342334466399465
time:  0.13341140747070312
27
Bicubic_x2:  29.541706253964676 ssim:  0.6841736948037443
Model4_x2:  33.93694395321057 ssim:  0.8026909869032448
Bicubic_x4:  25.96758677025616 ssim:  0.3798838150863553
Model4_x4:  28.49751401362644 ssim:  0.471673494780777

56
Bicubic_x2:  25.24466172937195 ssim:  0.601640818912144
Model4_x2:  29.28092569098782 ssim:  0.7612052407358827
Bicubic_x4:  20.351235239724645 ssim:  0.27507268580808525
Model4_x4:  21.937103059245665 ssim:  0.39474602987017793
time:  0.47873640060424805
57
Bicubic_x2:  24.4721207968448 ssim:  0.6132873595852635
Model4_x2:  28.488135330738736 ssim:  0.7629707612380506
Bicubic_x4:  20.48093087611107 ssim:  0.2906335573273605
Model4_x4:  22.177994997883165 ssim:  0.42920989181738617
time:  0.19673800468444824
58
Bicubic_x2:  25.78351457664843 ssim:  0.6747649880008127
Model4_x2:  28.703154799644793 ssim:  0.7861725751196026
Bicubic_x4:  21.081897332390817 ssim:  0.3233848109233287
Model4_x4:  22.324632290709708 ssim:  0.4193246949297854
time:  0.13270854949951172
59
Bicubic_x2:  26.20388391072529 ssim:  0.6694698157679106
Model4_x2:  28.704744427579264 ssim:  0.7920093177166735
Bicubic_x4:  22.832494639491543 ssim:  0.39485477373615135
Model4_x4:  24.135443724453953 ssim:  0.50658279

88
Bicubic_x2:  22.644959066913376 ssim:  0.587979055641974
Model4_x2:  24.502338065891752 ssim:  0.7000099483040492
Bicubic_x4:  18.635459063928767 ssim:  0.24074601878902846
Model4_x4:  19.559642884538754 ssim:  0.34188433793958306
time:  0.13675928115844727
89
Bicubic_x2:  28.877210593115386 ssim:  0.606627286568699
Model4_x2:  30.97052048171345 ssim:  0.7155611603639666
Bicubic_x4:  25.417077865102677 ssim:  0.3202604955172568
Model4_x4:  26.510241742313042 ssim:  0.4102603793349644
time:  0.15638995170593262
90
Bicubic_x2:  33.89487281473016 ssim:  0.6062314521725697
Model4_x2:  37.82283958865285 ssim:  0.7376814516951903
Bicubic_x4:  30.82076015323168 ssim:  0.29969490999622217
Model4_x4:  33.021123218726785 ssim:  0.3845417486556425
time:  0.5751330852508545
91
Bicubic_x2:  32.318431708101286 ssim:  0.674118380301885
Model4_x2:  41.65884722090002 ssim:  0.836505519419752
Bicubic_x4:  30.39009011422424 ssim:  0.3697363318506659
Model4_x4:  36.79984905169027 ssim:  0.5020382822851

In [ ]:
Bicubic_x2
PSNR:  30.916816407236617 SSIM:  0.7545968952428472
Model4_x2
PSNR:  31.435740014269793 SSIM:  0.7211516447513153
Bicubic_x4
PSNR:  28.414117980213348 SSIM:  0.48511500406746755
Model4_x4
PSNR:  28.186828598863183 SSIM:  0.4286470573069102